#### Bi-directional RNN example using tensorflow

In [1]:
'''
a bidirectional RNN/LSTM implementation using tensorflow
on MNIST dataset
'''
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

In [2]:
'''
- import dataset
'''
from tensorflow.examples.tutorials.mnist import input_data
mnist_dataset = input_data.read_data_sets('./mnist_dataset/', one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_dataset/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_dataset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist_dataset/t10k-images-idx3-ubyte.gz
Extracting ./mnist_dataset/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


to classify images using bidirectional LSTM, we consider every image row as sequence
of pixels. Because MNIST image shape is 28*28px, we will then handle 
28 sequences for 28 steps for every sample (to match 28x28)

In [3]:
'''
training parameters
'''
learning_rate = 0.001
epochs = 100
batch_size = 128
print_every = 200

In [4]:
'''
network parameters
'''
num_input = 28 # MNIST data input (img: 28)
timesteps = 28
num_hidden = 128
num_classes = 10 #0-9 of one hot encode

In [5]:
'''
placeholders for inputs and targets, weights and biases
'''
inputs = tf.placeholder("float32", [None, timesteps, num_input])
targets = tf.placeholder("float32", [None, num_classes])
'''
weights => 2*num_hidden because we have forward and backward
'''
weights = tf.Variable(tf.random_normal([2 * num_hidden, num_classes]))
bias = tf.Variable(tf.random_normal([num_classes]))


In [6]:
def BiRNN_model(inputs, weights, bias):
    '''
    - prepare data shape to match RNN function requirements
    - current data input shape: (batch_size, timesteps = 0, n_input)
    - required shape: 'timesteps' tensor list of shape (batch_size, num_input)
    - 28 of (128, 28)
    '''
    
    '''
    - unstack to get a list of 'timesteps' tensor of shape (batch_size, num_input)
    '''
    lstm_inputs = tf.unstack(inputs, timesteps, 1)
    
    '''
    - define both forward and backward lstm cell
    '''
    lstm_forward_cell = rnn.BasicLSTMCell(num_hidden, forget_bias = 1.0)
    lstm_backward_cell = rnn.BasicLSTMCell(num_hidden, forget_bias = 1.0)
    
    '''
    get output, no need final state
    '''
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_forward_cell,
                                                           lstm_backward_cell, 
                                                           lstm_inputs, dtype = tf.float32)
    #return as logits for softmax function
    #only use last output
    return tf.matmul(outputs[-1], weights) + bias 
    

In [7]:
logits = BiRNN_model(inputs, weights, bias)
predictions = tf.nn.softmax(logits)

In [8]:
'''
loss and optimizer, using logits above
'''
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                         logits = logits, labels = targets))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [9]:
'''
evaluate model
'''
correct_pred = tf.equal(tf.argmax(predictions, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [15]:
'''
-training
'''
epochs = 10000
with tf.Session() as sess:
    #variable initializer
    tf.global_variables_initializer().run()
    
    for epoch in range(epochs):
        batch_x, batch_y = mnist_dataset.train.next_batch(batch_size)
        '''
        - reshape batch_x to get seq of 28 x 28
        '''
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        #loss, final_state
        feed = {inputs: batch_x,
                targets: batch_y}
        loss, model_accuracy, _ = sess.run([cost, accuracy, optimizer], 
                                                  feed_dict = feed)
        
        if epoch % 100 == 0:
            print('epoch: {}/{} loss: {:.3f}, accuracy: {:.3f}'.
                  format(epoch, epochs, loss, model_accuracy))
    
    test_len = 10
    test_x = mnist_dataset.test.images[ : test_len].reshape((-1, timesteps, num_input))
    test_y = mnist_dataset.test.labels[ : test_len]
    feed = {inputs: test_x,
            targets: test_y}
    test_acc = sess.run(accuracy, feed_dict = feed)
    print('test accuracy: {}'.format(test_acc))
    
    sess.close()

epoch: 0/10000 loss: 2.975, accuracy: 0.047
epoch: 100/10000 loss: 2.285, accuracy: 0.172
epoch: 200/10000 loss: 2.142, accuracy: 0.211
epoch: 300/10000 loss: 1.959, accuracy: 0.406
epoch: 400/10000 loss: 2.068, accuracy: 0.367
epoch: 500/10000 loss: 1.897, accuracy: 0.414
epoch: 600/10000 loss: 1.775, accuracy: 0.438
epoch: 700/10000 loss: 1.681, accuracy: 0.562
epoch: 800/10000 loss: 1.615, accuracy: 0.562
epoch: 900/10000 loss: 1.725, accuracy: 0.461
epoch: 1000/10000 loss: 1.520, accuracy: 0.516
epoch: 1100/10000 loss: 1.578, accuracy: 0.539
epoch: 1200/10000 loss: 1.553, accuracy: 0.453
epoch: 1300/10000 loss: 1.318, accuracy: 0.641
epoch: 1400/10000 loss: 1.473, accuracy: 0.562
epoch: 1500/10000 loss: 1.377, accuracy: 0.609
epoch: 1600/10000 loss: 1.392, accuracy: 0.586
epoch: 1700/10000 loss: 1.550, accuracy: 0.508
epoch: 1800/10000 loss: 1.247, accuracy: 0.578
epoch: 1900/10000 loss: 1.263, accuracy: 0.594
epoch: 2000/10000 loss: 1.331, accuracy: 0.555
epoch: 2100/10000 loss: 1

In [14]:
print('finished')

finished
